In [1]:
import sys
import os
lib = r'D:\Technion studies\Keren Laboratory\python_playground\classifying-response-to-immunotherapy\utilities\droplet_dataset'
lib2 = r'D:\Technion studies\Keren Laboratory\python_playground\classifying-response-to-immunotherapy\utilities'
lib3 = r'D:\Technion studies\Keren Laboratory\python_playground\classifying-response-to-immunotherapy\data_analysis'
lib4 = r'D:\Technion studies\Keren Laboratory\python_playground\classifying-response-to-immunotherapy'
lib5 = r'D:\Technion studies\Keren Laboratory\python_playground\classifying-response-to-immunotherapy\scripts'
sys.path.append(lib)
sys.path.append(lib2)
sys.path.append(lib3)
sys.path.append(lib4)
sys.path.append(lib5)



In [2]:
import os
from os.path import join
import sklearn
from droplet_dataset import *
from utilities import *
from matplotlib import pyplot
import numpy as np
import scipy
import pickle
import matplotlib.pyplot as plt
import pickle
import pandas as pd
import random
from scipy.stats import pearsonr
from matplotlib.pyplot import figure
sklearn.__version__
%matplotlib notebook

In [11]:
def extract_sample(sample_id):
    sample_path = join(SAMPLES_PATH, sample_id, f'{sample_id}.pkl')
    rna_sample = pickle.load(open(sample_path, 'rb'))
    return rna_sample

def extract_sample_no_wrapper_folder(sample_id):
    sample_path = join(SAMPLES_PATH, sample_id)
    rna_sample = pickle.load(open(sample_path, 'rb'))
    return rna_sample


    
def get_informative_df(rna_sample):
    interesting_properties = ['is_apoptosis', 
                              'is_immune',
                              'is_cancer',
                              'is_stromal',
                              'cancer_immune_conflict', 
                              'is_doublet', 
                              'is_lymphoid',
                              'is_myeloid',
                              'should_be_removed',
                             'is_CelBender_empty']
    relevant_cells = {key:[cf for cf in rna_sample.cells_information if getattr(cf, key)] for key in interesting_properties}
    n_cells = len(relevant_cells)
    counter = {key1:{key2: 0 for key2 in interesting_properties} for key1 in interesting_properties}
    for prop1 in interesting_properties:
        for prop2 in interesting_properties:
            for cell_inf in relevant_cells[prop1]:
                counter[prop1][prop2] += int(getattr(cell_inf, prop2))
    df = pd.DataFrame(counter.values(), index=counter.keys())[counter.keys()]
    return  df

def check_cross_cell_types(sample_id):
    sample_path = join(SAMPLES_PATH, sample_id, f'{sample_id}.pkl')
    types = ['None', 'Immune_general', 'T cells', 'CD4 helper T cells', 'CD8 Cytotoxic T cells', 'Regulatory T cells',
             'Regulatory CD4 T cells', 'Regulatory CD8 T cells', 'Regulatory CD4_CD8 T cells', 'NKT cells', 'NK cells',
             'B cells', 'Activated T cells', 'Senescence T cells', 'Terminal effector', 'Exhausted T cells',
             'Stem_like T cells', 'Memory T cells', 'Memory CD4 T cells', 'Memory CD8 T cells',
             'Memory CD4_CD8 T cells', 'Macrophage_immature', 'Macrophage_mature', 'Monocyte_immature',
             'Monocyte_mature', 'cDCs_dendritic_cells', 'pDCs', 'myeloid cells_general_immature',
             'myeloid cells_general_mature', 'Neutrophils', 'Granolocytes']
    table = {k1: {k2: 0 for k2 in types} for k1 in types}
    data = pickle.load(open(sample_path, 'rb'))
    cls = [list(set(v)) for v in data.cells_information.getattr('cell_type_list')]
    cls = [v if len(v) else ['None'] for v in cls]

    for cell in cls:
        for t1 in cell:
            for t2 in cell:
                table[t1][t2] += 1
    df = pd.DataFrame(table.values(), index=table.keys())[table.keys()]
    return df

# Investigate properties of selected sample
Specify sample name and samples path

In [8]:
sample_id = 'M120'
ROW_SAMPLES_PATH = fr'D:\Technion studies\Keren Laboratory\Data\droplet_seq\ROW_DATA'
SAMPLES_INFORMATION_PATH = fr'D:\Technion studies\Keren Laboratory\python_playground\outputs\inferCNV\update_runs\4.3.21'


rna_sample = loading_sample(row_data_path=join(ROW_SAMPLES_PATH, f'{sample_id}.pkl'),
                                    cells_information_path=join(SAMPLES_INFORMATION_PATH, f'{sample_id}.pkl'))
rna_sample = rna_sample.filter_cells_by_property('should_be_removed', False)

In [9]:
get_informative_df(rna_sample)

,is_apoptosis,is_immune,is_cancer,cancer_immune_conflict,is_doublet,is_lymphoid,is_myeloid,should_be_removed
is_apoptosis,0,0,0,0,0,0,0,0
is_immune,0,1205,0,0,0,238,394,0
is_cancer,0,0,450,0,0,0,0,0
cancer_immune_conflict,0,0,0,0,0,0,0,0
is_doublet,0,0,0,0,0,0,0,0
is_lymphoid,0,238,0,0,0,238,163,0
is_myeloid,0,394,0,0,0,163,394,0
should_be_removed,0,0,0,0,0,0,0,0


# Investigate properties over all samples
Specify samples path and samples you're not interested in investigating them.

In [7]:
ROW_SAMPLES_PATH = fr'D:\Technion studies\Keren Laboratory\Data\droplet_seq\ROW_DATA'
SAMPLES_INFORMATION_PATH = fr'D:\Technion studies\Keren Laboratory\python_playground\outputs\inferCNV\update_runs\4.3.21'
not_investigate = []#['M133', 'M108']


In [9]:
samples = [subfolder for subfolder in os.listdir(SAMPLES_PATH) if subfolder.startswith('M') and (not subfolder in not_investigate)]
cross_check_df = None
for sample_id in samples:
#     print(sample)
    rna_sample = loading_sample(row_data_path=join(ROW_SAMPLES_PATH, f'{sample_id}'),
                                    cells_information_path=join(SAMPLES_INFORMATION_PATH, f'{sample_id}'))
    
    # REMOVING CELLS THAT SHOULD BE REMOVED
    rna_sample = rna_sample.filter_cells_by_property('should_be_removed', False)
    
    df = get_informative_df(rna_sample)
    if cross_check_df is None:
        cross_check_df = df
    else:
        cross_check_df = cross_check_df.add(df)

cross_check_df

,is_apoptosis,is_immune,is_cancer,cancer_immune_conflict,is_doublet,is_lymphoid,is_myeloid,should_be_removed
is_apoptosis,0,0,0,0,0,0,0,0
is_immune,0,89024,0,0,81,60145,18480,0
is_cancer,0,0,109283,12058,5256,0,0,0
cancer_immune_conflict,0,0,12058,12058,1154,0,0,0
is_doublet,0,81,5256,1154,5374,47,52,0
is_lymphoid,0,60145,0,0,47,60145,796,0
is_myeloid,0,18480,0,0,52,796,18480,0
should_be_removed,0,0,0,0,0,0,0,0


In [12]:
# If you want to save result
OUTPATH = fr'D:\Technion studies\Keren Laboratory\python_playground\outputs\investigate_cells_propeties\samples_properties_16.12.20_empty_removed.xlsx'
cross_check_df.to_excel(OUTPATH)

# Cross-checking cell-types of selected sample

In [48]:
SAMPLES_PATH = fr'D:\Technion studies\Keren Laboratory\python_playground\outputs\scrublet\5.12.20'
sample_id = 'M108'

In [52]:
check_cross_cell_types(sample_id).head()

,None,Immune_general,T cells,CD4 helper T cells,CD8 Cytotoxic T cells,Regulatory T cells,Regulatory CD4 T cells,Regulatory CD8 T cells,Regulatory CD4_CD8 T cells,NKT cells,...,Macrophage_immature,Macrophage_mature,Monocyte_immature,Monocyte_mature,cDCs_dendritic_cells,pDCs,myeloid cells_general_immature,myeloid cells_general_mature,Neutrophils,Granolocytes
None,7314,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Immune_general,0,2946,1398,135,705,30,6,0,0,57,...,3,24,3,87,120,0,0,135,651,159
T cells,0,1398,1398,135,705,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CD4 helper T cells,0,135,135,135,15,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CD8 Cytotoxic T cells,0,705,705,15,705,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Cross-checking cell-types over all samples

In [9]:
SAMPLES_PATH = fr'D:\Technion studies\Keren Laboratory\python_playground\outputs\scrublet\16.12.20_empty_removed'
not_investigate = [] #['M133', 'M108']

In [14]:
samples = [subfolder for subfolder in os.listdir(SAMPLES_PATH) if subfolder.startswith('M') and (not subfolder in not_investigate)]
cell_types_df = None
for sample in samples:
#     print(sample)
    df = check_cross_cell_types(sample)
    if cell_types_df is None:
        cell_types_df = df
    else:
        cell_types_df = cell_types_df.add(df)

cell_types_df

,None,Immune_general,T cells,CD4 helper T cells,CD8 Cytotoxic T cells,Regulatory T cells,Regulatory CD4 T cells,Regulatory CD8 T cells,Regulatory CD4_CD8 T cells,NKT cells,...,Macrophage_immature,Macrophage_mature,Monocyte_immature,Monocyte_mature,cDCs_dendritic_cells,pDCs,myeloid cells_general_immature,myeloid cells_general_mature,Neutrophils,Granolocytes
None,164285,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Immune_general,0,50565,24250,2358,12195,454,257,37,28,1112,...,120,1936,216,3739,2327,54,44,3249,9917,583
T cells,0,24250,24250,2358,12195,2,1,0,0,64,...,0,0,4,0,0,0,0,0,0,0
CD4 helper T cells,0,2358,2358,2358,134,0,1,0,0,5,...,0,0,2,0,0,0,0,0,0,0
CD8 Cytotoxic T cells,0,12195,12195,134,12195,0,0,0,0,32,...,0,0,1,0,0,0,0,0,0,0
Regulatory T cells,0,454,2,0,0,454,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
Regulatory CD4 T cells,0,257,1,1,0,0,257,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Regulatory CD8 T cells,0,37,0,0,0,0,0,37,0,0,...,0,0,0,0,0,0,0,0,0,0
Regulatory CD4_CD8 T cells,0,28,0,0,0,0,0,0,28,0,...,0,0,0,0,0,0,0,0,0,0
NKT cells,0,1112,64,5,32,1,0,0,0,1112,...,0,0,0,0,0,0,0,0,0,0


In [1]:
# If you want to save result
# OUTPATH = fr'D:\Technion studies\Keren Laboratory\python_playground\outputs\investigate_cells_propeties\samples_cell_types_16.12.20_empty_removed.xlsx'
# cell_types_df.to_excel(OUTPATH)

# DEBUG

In [12]:
ROW_SAMPLES_PATH = fr'D:\Technion studies\Keren Laboratory\Data\droplet_seq\ROW_DATA'
SAMPLES_INFORMATION_PATH = fr'D:\Technion studies\Keren Laboratory\python_playground\outputs\inferCNV\update_runs\4.3.21'
not_investigate = []#['M133', 'M108']
samples = [subfolder for subfolder in os.listdir(ROW_SAMPLES_PATH) if subfolder.startswith('M') and (not subfolder in not_investigate)]
cross_check_df = None
for sample_id in samples:
#     print(sample)
    rna_sample = loading_sample(row_data_path=join(ROW_SAMPLES_PATH, f'{sample_id}'),
                                    cells_information_path=join(SAMPLES_INFORMATION_PATH, f'{sample_id}'))
    
    # REMOVING CELLS THAT SHOULD BE REMOVED
    rna_sample = rna_sample.filter_cells_by_property('should_be_removed', False)
    
    df = get_informative_df(rna_sample)
    if cross_check_df is None:
        cross_check_df = df
    else:
        cross_check_df = cross_check_df.add(df)

cross_check_df

,is_apoptosis,is_immune,is_cancer,is_stromal,cancer_immune_conflict,is_doublet,is_lymphoid,is_myeloid,should_be_removed,is_CelBender_empty
is_apoptosis,0,0,0,0,0,0,0,0,0,0
is_immune,0,89024,0,0,0,81,60145,18480,0,0
is_cancer,0,0,109283,0,12058,5256,0,0,0,0
is_stromal,0,0,0,28177,0,37,0,0,0,0
cancer_immune_conflict,0,0,12058,0,12058,1154,0,0,0,0
is_doublet,0,81,5256,37,1154,5374,47,52,0,0
is_lymphoid,0,60145,0,0,0,47,60145,796,0,0
is_myeloid,0,18480,0,0,0,52,796,18480,0,0
should_be_removed,0,0,0,0,0,0,0,0,0,0
is_CelBender_empty,0,0,0,0,0,0,0,0,0,0


In [13]:
SAMPLES_INFORMATION_PATH = fr'D:\Technion studies\Keren Laboratory\python_playground\outputs\inferCNV\update_runs\10.5.21'
not_investigate = []#['M133', 'M108']
samples = [subfolder for subfolder in os.listdir(ROW_SAMPLES_PATH) if subfolder.startswith('M') and (not subfolder in not_investigate)]
cross_check_df = None
for sample_id in samples:
#     print(sample)
    rna_sample = loading_sample(row_data_path=join(ROW_SAMPLES_PATH, f'{sample_id}'),
                                    cells_information_path=join(SAMPLES_INFORMATION_PATH, f'{sample_id}'))
    
    # REMOVING CELLS THAT SHOULD BE REMOVED
    rna_sample = rna_sample.filter_cells_by_property('should_be_removed', False)
    
    df = get_informative_df(rna_sample)
    if cross_check_df is None:
        cross_check_df = df
    else:
        cross_check_df = cross_check_df.add(df)

cross_check_df

,is_apoptosis,is_immune,is_cancer,is_stromal,cancer_immune_conflict,is_doublet,is_lymphoid,is_myeloid,should_be_removed,is_CelBender_empty
is_apoptosis,0,0,0,0,0,0,0,0,0,0
is_immune,0,84848,0,0,0,47,59349,14304,0,34357
is_cancer,0,0,97225,0,0,4102,0,0,0,14707
is_stromal,0,0,0,31557,0,38,0,3380,0,9661
cancer_immune_conflict,0,0,0,0,0,0,0,0,0,0
is_doublet,0,47,4102,38,0,4187,14,19,0,0
is_lymphoid,0,59349,0,0,0,14,59349,0,0,24160
is_myeloid,0,14304,0,3380,0,19,0,17684,0,4858
should_be_removed,0,0,0,0,0,0,0,0,0,0
is_CelBender_empty,0,34357,14707,9661,0,0,24160,4858,0,58725


In [14]:
SAMPLES_INFORMATION_PATH = fr'D:\Technion studies\Keren Laboratory\python_playground\outputs\inferCNV\update_runs\10.5.21'
not_investigate = []#['M133', 'M108']
samples = [subfolder for subfolder in os.listdir(ROW_SAMPLES_PATH) if subfolder.startswith('M') and (not subfolder in not_investigate)]
cross_check_df = None
for sample_id in samples:
#     print(sample)
    rna_sample = loading_sample(row_data_path=join(ROW_SAMPLES_PATH, f'{sample_id}'),
                                    cells_information_path=join(SAMPLES_INFORMATION_PATH, f'{sample_id}'))
    
    # REMOVING CELLS THAT SHOULD BE REMOVED
#     rna_sample = rna_sample.filter_cells_by_property('should_be_removed', False)
    
    df = get_informative_df(rna_sample)
    if cross_check_df is None:
        cross_check_df = df
    else:
        cross_check_df = cross_check_df.add(df)

cross_check_df

,is_apoptosis,is_immune,is_cancer,is_stromal,cancer_immune_conflict,is_doublet,is_lymphoid,is_myeloid,should_be_removed,is_CelBender_empty
is_apoptosis,49517,1708,27138,13622,1507,4978,639,350,49517,11431
is_immune,1708,91679,0,0,0,1445,62428,17532,6831,36282
is_cancer,27138,0,124546,0,0,7891,0,0,27321,19387
is_stromal,13622,0,0,45381,0,1318,31,3411,13824,13485
cancer_immune_conflict,1507,0,0,0,32212,2189,0,0,32212,4760
is_doublet,4978,1445,7891,1318,2189,12874,1110,255,8687,460
is_lymphoid,639,62428,0,31,0,1110,62459,848,3110,25153
is_myeloid,350,17532,0,3411,0,255,848,20943,3259,5549
should_be_removed,49517,6831,27321,13824,32212,8687,3110,3259,85730,16954
is_CelBender_empty,11431,36282,19387,13485,4760,460,25153,5549,16954,75679


In [21]:
path = fr'D:\Technion studies\Keren Laboratory\python_playground\outputs\classifying_cell_types\5.12.20'
sample_id = r'M132'
sample_path = join(path, sample_id, f'{sample_id}.pkl')
data = pickle.load(open(sample_path, 'rb'))

In [7]:
samples = [subfolder for subfolder in os.listdir(SAMPLES_PATH) if subfolder.startswith('M') and (not subfolder in not_investigate)]
all_df = None
count = 0
for sample in samples:
    print(sample)
    sample_path = join(SAMPLES_PATH, sample, f'{sample}.pkl')
    data = pickle.load(open(sample_path, 'rb'))
    xx = sum([c.is_immune for c in data.cells_information])
    print(xx)
    count += xx
    
    yy = [1 for c in data.cells_information if c.is_immune == False and  len(c.cell_type_list)>0]
    print(sum(yy))
print(count)

M100
190
0
M101
1467
0
M102
734
0
M103
433
0
M104
277
0
M105
3094
0
M106
562
0
M107
1627
0
M108
116
0
M109
2121
0
M110
1869
0
M111
5658
0
M112
178
0
M114
4530
0
M115
2910
0
M116
3380
0
M118
1023
0
M120
1219
0
M121
4071
0
M122
676
0
M123
3368
0
M124
4274
0
M125
140
0
M126
2757
0
M127
78
0
M128
422
0
M129
527
0
M130
3273
0
M131
3986
0
M132
2663
0
M133
83
0
M134
1798
0
M135
1955
0
M136
1268
0
M137
3758
0
M138
2428
0
M139
2428
0
M140
3395
0
M141
85
0
M143
1168
0
M144
538
0
M145
3983
0
M146
5783
0
M97
1299
0
M98
760
0
M99
3296
0
91648


In [15]:
import pandas as pd
import numpy as np

In [29]:
df = pd.DataFrame(np.array([[1,2,3,4], ['One', 'Two', 'Three', 'Four']]).T, columns=['Num', 'word'])
df

,Num,word
0,1,One
1,2,Two
2,3,Three
3,4,Four


In [28]:
df.iloc[[0], [0, 1]]

,Num,word
0,1,One


In [28]:
df.iloc[[0], [0, 1]]

,Num,word
0,1,One


In [26]:
~np.array([True, False])

array([False,  True])